# Run SWMM simulations

###  Import packages 

In [3]:
import itertools
import pandas as pd
import numpy as np
import pickle
from itertools import combinations

from pyswmm import Simulation, Nodes
from pyswmm import Simulation, Subcatchments, SystemStats
import matplotlib.pyplot as plt
import seaborn as sns
import os,sys
import matplotlib.pyplot as plt;
import pickle


from pyswmm import Simulation;
from pyswmm import Subcatchments;
from pyswmm import Nodes;
from pyswmm import SystemStats;
from pyswmm import Simulation;
from swmmtoolbox import swmmtoolbox;
from itertools import combinations
from os.path import dirname


parent_dir = (dirname(os.getcwd()))
current_dir = os.getcwd()
sys.path.append("\\".join([parent_dir, 'functions']) )

for mod in ['swmm_sims', 'swmm_cal']:
    if mod in sys.modules: 
        del sys.modules[mod]
        
import swmm_sims
import swmm_cal

from swmm_sims import *   
from swmm_cal import *

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

### Import sve runs


In [4]:
file = os.path.join(parent_dir, 'inp_files\\SVE_FILES\\sve_core.pickle')
sve_core = pd.read_pickle(file)

In [5]:
#generate soils dict
soils_dict={'Ks':[20,50,80], #K in mm/hr
        'H_i':[0,0,0], #initial head, mm
        'IMD':[0,0,0]
        }  

soil = pd.DataFrame(soils_dict)

### Generate SWMM calibration parameter sets

In [6]:
# run two_params function
two_params(sve_core,100)
file = os.path.join(parent_dir, 'out\\2params.pickle')

#open params pickle
with open(file, 'rb') as handle:
    params = pickle.load(handle)
    params_df = pd.DataFrame(params)

###  Run a test model to make sure .inp file is getting updated  

In [9]:
# enter inputs here
P=2
folder='cal'
# dur='30min'
dstor_line = 63
perm_line = 71

#run path function to define write, read, and output file paths
write_path,read_path,out_path,run=paths(P,parent_dir,'SWMM_inp\\cal')
    
print(write_path)

# set some random parameter values to update to
n_imperv= 0.01
n_perv=0.1
d_imperv=0
d_perv=0
Ks=50
H_i=0
IMD=0

print("new parameters -->         S1               ", n_imperv,"       ", n_perv,"       ", d_imperv,"       ",d_perv)

#step through input and print the line of the .inp file to be updated
update_inp(read_path,write_path,n_imperv,n_perv,d_imperv,d_perv,Ks,H_i,IMD)

with open(write_path) as f:
    for i, line in enumerate(f, 1):
        if i ==perm_line:
            break
print(line)         

C:\Users\annel\OneDrive\Desktop\SWMM_SVE\FINAL_FILES_v2\inp_files\SWMM_inp\cal\P2_WRITE.inp
new parameters -->         S1                0.01         0.1         0         0
S2               0	    50         0



### Now run all parameters and update .inp file each time

In [ ]:
#run for all Ps, Kss
Ps = [2,5,8]
dur='30min'
folder='cal'
for i,p in enumerate(Ps):
    P=p
    print(P)
    # set paths (write and read inp files, out folder)
    write_path,read_path,out_path,run=paths(P,parent_dir,'SWMM_inp\\cal')
    # test simulation on write path
    test_sim(write_path)

    import time
    start = time.time()

    perm_line = 71
    data1=dict()
    j=len(soil)
    g=dict()
    data=pd.DataFrame()
    data2df=pd.DataFrame()

    if P==2:
        params=params_df.loc[params_df['p']==2.0].copy()
    if P==5:
        params=params_df.loc[params_df['p']==5.0].copy()
    else:
        params=params_df.loc[params_df['p']==8.0].copy()

    for index, row in params.iterrows():

        data2df=data2df
        A1=row['A1']
        A2=row['A2']
        A3=row['A3']
        W1=row['W1']
        W2=row['W2']
        W3=row['W3']
        Adc_frac=row['Adc_frac']
        fV=row['fV']
        A_imp=row['A_imp']
        image=row['image']
        n_imperv= 0.01
        n_perv=0.1
        d_imperv=0
        d_perv=0
    
        for i in range(j):
            data=pd.DataFrame()
            Ks=soil["Ks"][i] #    pulls the K value from soil dictionary
            H_i=soil["H_i"][i] #    suction head
            IMD=soil["IMD"][i] #    initial moisture deficit

            update_inp(read_path,write_path,n_imperv,n_perv,d_imperv,d_perv,Ks,H_i,IMD)
            
            from time import sleep
            for x in range(0, 4):  # try 4 times

                try:
                    infiltration,runoff,rainfall,stor=run_sim_cal(write_path,A1,A2,A3,W1,W2,W3,1,P)
                    # msg.send()

                    str_error = None

                except:
                    str_error = True
    #                 pass

                if str_error==True:
                    sleep(10)  # wait for 2 seconds before trying to fetch the data again
                else:  
                    break



            data.set_value(i,'infiltration',infiltration)
            data.set_value(i,'runoff',runoff)
            data.set_value(i,'rainfall_mm',rainfall)
            data.set_value(i,'stor_mm',stor)

            data.set_value(i,'param',index)
            data.set_value(i,'A1',A1)
            data.set_value(i,'A2',A2)
            data.set_value(i,'A3',A3)
            data.set_value(i,'Adc_frac',Adc_frac)
            data.set_value(i,'fV',fV)
            data.set_value(i,'A_imp',A_imp)
            data.set_value(i,'W',W1)
            data.set_value(i,'P',P)
            data.set_value(i,'H_i',H_i)
            data.set_value(i,'Ks',Ks)
            data.set_value(i,'IMD',IMD)
            data.set_value(i,'soil',i)
            data.set_value(i,'image',image)
            data1.update(data)   

            data1df=pd.DataFrame(data1)
            frames1=[data1df,data2df]
            data2df=pd.concat(frames1)  
            data2df['run']=run


    data2df['Ks']=(data2df['Ks']/10).astype(float).round(1)
    data2df=data2df.reset_index()
    for key in data2df.index:
        sim = data2df.loc[key]
        scenario = "p-{0},Ks-{1}".format(sim.P,sim.Ks)
        data2df.loc[key,"scenario"] = scenario

    data2df['run']=run
    data2df['total']=data2df['infiltration']+data2df['runoff']
    data2df['IF']=data2df['infiltration']/(data2df['total'])

    name = '\\'.join([parent_dir, 'out\\cal\\'+str(run)+'.pickle'])
    with open(name, 'wb') as handle:
        pickle.dump(data2df, handle, protocol=pickle.HIGHEST_PROTOCOL)

    end = time.time()
    print(end - start)

2
{'drains': 0.0, 'evaporation': 0.0, 'final_snow_cover': 0.0, 'final_storage': 0.0, 'infiltration': 597.0627148442712, 'init_snow_cover': 0.0, 'init_storage': 0.0, 'routing_error': 0.0, 'rainfall': 9.999999999999996, 'runoff': 500.30632068085043, 'runon': 0.0, 'snow_removed': 0.0}
